# Imports

In [1]:
# Import necessary modules
import os
from lib.streaming.streaming import run_streaming

# Variables

In [2]:
# input_fastq_filename = "test-combined.fastq"
input_fastq_filename = "rbcL_Qiagen_tomato_5000.fastq"
windows = False
db = "matK"

In [3]:
input_fastq_path = os.path.join("assets","input", input_fastq_filename)
base_name = os.path.splitext(input_fastq_filename)[0]

# Streaming Pipeline

In [4]:
# choose a streaming method between the following:
# - "basic_streaming"

streaming_method = "basic_streaming"

# choose a consensus method between the following:
# - "80_20_best_sequence"
# - "80_20_longest_sequence"
# - "straightforward_best_sequence"

consensus_method = "80_20_best_sequence"

# choose an identification method between the following:
# - "blastn"

identification_method = "blastn"

# define the streaming parameters
species_identification_percentage_dominance = 0.7
minimum_block_amount_before_dominance_check = 5
block_size = 250

run_streaming(input_name= base_name, 
              input_fastq_path= input_fastq_path, 
              streaming_method= streaming_method,
              db= db,
              windows= windows,
              species_identification_percentage_dominance=species_identification_percentage_dominance,
              block_size=block_size,
              minimum_block_amount_before_dominance_check=minimum_block_amount_before_dominance_check,
              consensus_method=consensus_method,
              identification_method=identification_method)

Logging set up at assets/output/streaming/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_streaming_pipeline_log.log
Block 1 saved at assets/output/streaming/rbcL_Qiagen_tomato_5000/block_1
Block 2 saved at assets/output/streaming/rbcL_Qiagen_tomato_5000/block_2
Block 3 saved at assets/output/streaming/rbcL_Qiagen_tomato_5000/block_3
Block 4 saved at assets/output/streaming/rbcL_Qiagen_tomato_5000/block_4
Block 5 saved at assets/output/streaming/rbcL_Qiagen_tomato_5000/block_5
Block 6 saved at assets/output/streaming/rbcL_Qiagen_tomato_5000/block_6
